# Lab 2 - Exploring Text Data

### Eric Smith and Jake Carlson

## Introduction
For this lab we will be examining questions and answers on the popular programming Q&A website, Stack Overflow. Stack Overflow allows programmers who are stuck on a problem ask the community about ways to resolve or circumvent said problem. This helps people write more accurate code faster. However, the community can be pretty hard on those who do not fully understand their question or are disrespectful. It could also be the case that the few developers who have experience with a problem similar to yours are not on the website when you post your question. If too much time passes, your post may be burried forever. This makes for some interesing questions. When should you make your post so that it has the highest chance of being answered? Are there specific keywords or phasing that encourage other programmers to answer your questions? Are there specific languages that have their questions answered faster than others? These are questions we will be exploring int his lab.

## Business Understanding

### Motivations
Stack Overflow is a go-to resource for developers. Questions are often distilled down to a single block of code that is easily digestible, and can be matched to the question you came to the site for. However, if you're truly stuck on something, it can seem like an eternity of waiting before someone responds to your question, and sometimes, it may not be answered at all. If you're working in industry and spending a lot of time waiting for a question to be answered, you could fall behind schedule and miss deadlines. These delays could cost your company a great deal in lost revenue and tech debt. Therefore, accurately predicting the amount of time to get a question answered can be seen as a valuable project management tool.

### Objectives
It would be useful if you could predict how quickly your question will be answered if you post it at various times through out the day. For a prediction tool like this to be useful, we want something that is 80-95% accurate in predicting when a question will be answered by. As a project manager, you want your developers to be asking questions that are concise and respectful. Therefore, a tool that could recommend keywords to add to your post to make it more attractive, and provide you with approximate gains in time for reformatting, would also be a valuable project management tool.

## Data Understanding

### Data Attributes
The following is a list of attributes in the data, their data types, and a brief description of the attribute.

#### Questions
- **Id** (nominal): A unique identifier for each question
- **OwnerUserId** (nominal): A unique identifier for the person who posted the question
- **CreationDate** (ordinal): A timestamp of when the question was posted
- **ClosedDare** (ordinal): A timestamp of when the question was closed, if the question wasn't closed this field is NaN
- **Score** (ordinal): ratio?? The number of upvotes a post has
- **Title** (text): A title for the question
- **Body** (text): The question body

#### Answers
- **Id** (nominal): A unique identifier for each answer
- **OwnerUserId** (nominal): A unique identifier for the person who posted the answer
- **CreationDate** (ordinal): A timestamp of when the answer was posted
- **ParentId** (nominal): The unique identifier of the question this answer is for
- **Score** (ordinal): ratio?? The number of upvotes an answer has
- **Body** (text): The answer body

#### Tags
- **Id** (nominal): The *Id* of the question the tag is associated with
- **Tag** (nominal): The tag a question has been assigned

## Data Quality
Our data it too big, so we will take a rendom subsample of 40% of the original data set.

In [92]:
import numpy as np
import pandas as pd

In [93]:
# read data
df = pd.read_csv('./data/Questions.csv', encoding='ISO-8859-1')

In [94]:
df.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1264216 entries, 0 to 1264215
Data columns (total 7 columns):
Id              1264216 non-null int64
OwnerUserId     1249762 non-null float64
CreationDate    1264216 non-null object
ClosedDate      55959 non-null object
Score           1264216 non-null int64
Title           1264216 non-null object
Body            1264216 non-null object
dtypes: float64(1), int64(2), object(4)
memory usage: 67.5+ MB


Trial and error has shown us that 1,264,000 elements is too many to work with. Therefore, we will use a random sample to reduce our data set size to 12,000 elements.

In [96]:
df = df.sample(n=12000, replace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 929155 to 177173
Data columns (total 7 columns):
Id              12000 non-null int64
OwnerUserId     11843 non-null float64
CreationDate    12000 non-null object
ClosedDate      564 non-null object
Score           12000 non-null int64
Title           12000 non-null object
Body            12000 non-null object
dtypes: float64(1), int64(2), object(4)
memory usage: 750.0+ KB


The body of each question is formatted in HTML, so we will need to parse through each body and remove the HTML tags. We also need to replace HTML entities with their character representations. Finally, many questions have accompanying code blocks, so we will parse out the code blocks into a new attribute for each entry.

In [97]:
from bs4 import BeautifulSoup
import html
import re

# compile regexes
tag_re = re.compile('<[^<]+?>')
newline_re = re.compile('\n+')

def clean_html(body_text):
    soup = BeautifulSoup(body_text, 'html.parser')
    code_tags = soup.findAll('code')
    code_text = ""
    
    # unescape html entities
    body_text = html.unescape(body_text)
    
    # remove code blocks, saving blocks to new string
    for c in code_tags:
        body_text = body_text.replace("<code>{}</code>".format(c.string),
                                      '')
        if c.string:
            code_text += c.string

    # remove remaining tags and multiple newlines
    body_text = tag_re.sub('', body_text)
    body_text = newline_re.sub('\n', body_text)
    
    return body_text, code_text

as_list = df.Body.tolist()
body_list = []
code_list = []
for i in range(len(as_list)):
    body_text, code_text = clean_html(as_list[i])
    body_list.append(body_text)
    code_list.append(code_text)

# update body column
df = df.assign(Body=body_list)
# add code column
df = df.assign(Code=code_list)
df.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,Code
929155,30970580,1551974.0,2015-06-22T00:10:40Z,NaN,0,Left click not working on notifyicon with cont...,I have a tray application and I would like to ...,private void notifyIcon1_MouseClick(object...
70165,3124480,200567.0,2010-06-26T15:56:43Z,NaN,1,How to have a UIView appear from the middle of...,"I looked at Apple's UIModalTransitionStyle, bu...",
942803,31381300,4810042.0,2015-07-13T10:46:01Z,NaN,-2,How to iterate foreach loop for a particular a...,I'm having two different for-each loop as belo...,"<div id=""filterarea"" data-bind=""foreach: { da..."
562743,19463330,1102566.0,2013-10-19T06:54:14Z,NaN,4,Using Reactive cocoa to observe UITouches?,Im fairly new to reactive cocoa and im trying ...,@interface MyView:UIView{\n\n UITouch *...
769139,26028280,3267279.0,2014-09-24T23:44:32Z,NaN,2,Not understanding seg faults when I'm using po...,I keep getting seg faults and I can't for the ...,"int main()\n{\n int i1, i2;\n struct int..."


With the HTML tags removed and the code isolated, the question body is much cleaner. Now lets add another column for the time in minutes it took to get the question answered. For this value, we will look for the highest rated answer for each question and use the time when that answer was posted.

In [98]:
df_ans = pd.read_csv('./data/Answers.csv', encoding='ISO-8859-1')
df_ans.head()

,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,92,61.0,2008-08-01T14:45:37Z,90,13,"<p><a href=""http://svnbook.red-bean.com/"">Vers..."
1,124,26.0,2008-08-01T16:09:47Z,80,12,<p>I wound up using this. It is a kind of a ha...
2,199,50.0,2008-08-01T19:36:46Z,180,1,<p>I've read somewhere the human eye can't dis...
3,269,91.0,2008-08-01T23:49:57Z,260,4,"<p>Yes, I thought about that, but I soon figur..."
4,307,49.0,2008-08-02T01:49:46Z,260,28,"<p><a href=""http://www.codeproject.com/Article..."


In [99]:
df_ans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2014516 entries, 0 to 2014515
Data columns (total 6 columns):
Id              int64
OwnerUserId     float64
CreationDate    object
ParentId        int64
Score           int64
Body            object
dtypes: float64(1), int64(3), object(2)
memory usage: 92.2+ MB


In [100]:
ids = df.Id.unique()
df_ans = df_ans[df_ans.ParentId.isin(ids)]
df_ans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19157 entries, 83 to 2014494
Data columns (total 6 columns):
Id              19157 non-null int64
OwnerUserId     19035 non-null float64
CreationDate    19157 non-null object
ParentId        19157 non-null int64
Score           19157 non-null int64
Body            19157 non-null object
dtypes: float64(1), int64(3), object(2)
memory usage: 1.0+ MB


We will clean the answers data frame using the same function as the questions data frame.

In [101]:
as_list = df_ans.Body.tolist()
body_list = []
code_list = []
for i in range(len(as_list)):
    body_text, code_text = clean_html(as_list[i])
    body_list.append(body_text)
    code_list.append(code_text)

# update body column
df_ans = df_ans.assign(Body=body_list)
# add code column
df_ans = df_ans.assign(Code=code_list)
df_ans.head()

,Id,OwnerUserId,CreationDate,ParentId,Score,Body,Code
83,2699,36.0,2008-08-05T18:53:44Z,2630,0,"While semi-related to your question, it does n...",
160,4692,615.0,2008-08-07T13:11:06Z,2630,2,I wrote a provider to give me access to 's f...,PowerShellIE7RSScdPowerShellCodePlex
184,5498,49.0,2008-08-08T00:02:35Z,2630,8,there's an out-twitter script i use for postin...,
341,10160,1233.0,2008-08-13T18:19:07Z,2630,2,"While it is not as fun as Out-Twitter, my favo...",
436,11452,1358.0,2008-08-14T18:20:02Z,2630,1,"Well it is a little bland, but I would vote fo...",


In [102]:
# adapted from https://stackoverflow.com/questions/2788871/date-difference-in-minutes-in-python
from datetime import datetime
import time

fmt = '%Y-%m-%dT%H:%M:%SZ'
def get_minutes_diff(timestamp1, timestamp2):
#     if np.isnan(x.ClosedDate):
#         return np.nan
    d1 = datetime.strptime(timestamp1, fmt)
    d2 = datetime.strptime(timestamp2, fmt)
    
    # convert to unix timestamp
    d1_ts = time.mktime(d1.timetuple())
    d2_ts = time.mktime(d2.timetuple())
    
    # convert to minutes
    return int(d2_ts - d1_ts) / 60

durations = []
for i, row in df.iterrows():
    answer = df_ans[ df_ans.ParentId == row.Id ]
    answer = answer[ answer.Score == answer.Score.max() ]
    answer_time = answer.CreationDate.values
    if len(answer_time) > 0:
        answer_time = answer_time[0]
        dur = get_minutes_diff(row.CreationDate, answer_time)
        if dur > -1: 
            df.loc[i, 'Duration'] = dur

df.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,Code,Duration
929155,30970580,1551974.0,2015-06-22T00:10:40Z,NaN,0,Left click not working on notifyicon with cont...,I have a tray application and I would like to ...,private void notifyIcon1_MouseClick(object...,15.650000
70165,3124480,200567.0,2010-06-26T15:56:43Z,NaN,1,How to have a UIView appear from the middle of...,"I looked at Apple's UIModalTransitionStyle, bu...",,328498.916667
942803,31381300,4810042.0,2015-07-13T10:46:01Z,NaN,-2,How to iterate foreach loop for a particular a...,I'm having two different for-each loop as belo...,"<div id=""filterarea"" data-bind=""foreach: { da...",43.316667
562743,19463330,1102566.0,2013-10-19T06:54:14Z,NaN,4,Using Reactive cocoa to observe UITouches?,Im fairly new to reactive cocoa and im trying ...,@interface MyView:UIView{\n\n UITouch *...,3455.700000
769139,26028280,3267279.0,2014-09-24T23:44:32Z,NaN,2,Not understanding seg faults when I'm using po...,I keep getting seg faults and I can't for the ...,"int main()\n{\n int i1, i2;\n struct int...",3.566667


Load the tags and save the tags that correspond to a question.

In [103]:
df_tags = pd.read_csv('./data/Tags.csv', encoding='ISO-8859-1')
df_tags.head()

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


In [104]:
ids = df.Id.unique()
df_tags = df_tags[df_tags.Id.isin(ids)]
df_tags.head()

,Id,Tag
88,2630,powershell
89,2630,cmdlets
420,14530,linq
421,14530,linq-to-sql
422,14530,stored-procedures


In [105]:
# http://www.labautopedia.org/mw/List_of_programming_and_computer_science_terms
topics = pd.read_csv('./data/CS_terms.csv', encoding='ISO-8859-1')
list_topics = [x.upper() for x in topics.Term.tolist()]

Save the cleaned data to a persistance file.

## Data Visualization

In [ ]:
df.to_csv('./clean-data/Questions.csv', sep=',')
df_ans.to_csv('./clean-data/Answers.csv', sep=',')
df_tags.to_csv('./clean-data/Tags.csv', sep=',')

In [ ]:
# tf-idf for most common words that appear in the body

import re
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(encoding='ISO-8859-1') 

tfidf_mat = tfidf_vect.fit_transform( df.Body.tolist() )

tfidf_mat = pd.DataFrame(data=tfidf_mat.toarray(),columns=tfidf_vect.get_feature_names())

common_words = tfidf_mat[ [col for col in tfidf_mat.columns if col.upper() in list_topics] ]
most_common_words = common_words.max().sort_values()[-10:]

In [ ]:
most_common_words.plot(kind='barh', fontsize=12, figsize=(8,6))
plt.xlabel('Percentage of Occurances', fontsize=14)
plt.title('TF-IDF: Most Common Words', fontsize=18)
plt.legend([])
plt.show()

In [ ]:
df = pd.read_csv('./clean-data/Questions.csv')
df_ans = pd.read_csv('./clean-data/Answers.csv')
df_tags = pd.read_csv('./clean-data/Tags.csv')

In [ ]:
df_tags.info()

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter

tag_counts = Counter(df_tags.Tag)

# remove tags that have less than 175 occurances
for t in list(tag_counts):
    if tag_counts[t] < 175:
        del tag_counts[t]

df_tag_counts = pd.DataFrame.from_dict(tag_counts, orient='index')
df_tag_counts.plot(kind='barh', fontsize=12, figsize=(8,6))
plt.xlabel('Number of occurances', fontsize=14)
plt.title('Tags', fontsize=18)
plt.legend([])
plt.show()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(stop_words='english')
bag_words = count_vect.fit_transform(df.Title.tolist())

In [ ]:
len(count_vect.vocabulary_)

In [ ]:
df_title = pd.DataFrame(data=bag_words.toarray(), 
                        columns=count_vect.get_feature_names())

In [ ]:
# print 10 most common words
df_title.sum().sort_values()[-10:]

In [ ]:
# adapted from https://stackoverflow.com/a/16359854
# 1 day = 86400 seconds
# 1 week = 604800 seconds
import sys

# put the durations into different buckets, graph the result
# only count the durations that are not null
df_durations = pd.cut(df.Duration[ df.Duration.notnull() ], 
                      bins=[-1, 3600, 86400, 604800, 2592000, sys.maxsize], 
                      labels=False)
# labels correspond to buckets of time above
labels = np.array(['Less than 1 hour', 'Less than 1 day', 'Less than 1 week', 'More than 1 week', 'More than 1 month'])
df_durations = labels[df_durations]

duration_counts = Counter(df_durations) # count each occurance

In [ ]:
df_duration_counts = pd.DataFrame.from_dict(duration_counts, orient='index')
df_duration_counts.plot(kind='barh', fontsize=12, figsize=(8,6))
plt.xlabel('Number of Occurances', fontsize=14)
plt.title('Duration Till Answered', fontsize=18)
plt.legend([])
plt.show()

In [ ]:
df.Code[1]

# put the code into different buckets, graph the result
# count each occurance of a '\n' as a newline
df_lines_of_code = pd.cut( [x.count('\n') for x in df.Code[ df.Code.notnull() ] ], 
                       bins=[-1, 5, 10, 20, sys.maxsize],
                       labels=False)

# labels correspond to buckets of length above
labels = np.array(['Less than 5', '6-10', '11-20', '20+'])
df_lines_of_code = labels[df_lines_of_code]

lines_of_code_count = Counter(df_lines_of_code)

In [ ]:
df_lines_of_code_count = pd.DataFrame.from_dict(lines_of_code_count, orient='index')
df_lines_of_code_count.plot(kind='barh', fontsize=12, figsize=(8,6))
plt.xlabel('Number of Occurances', fontsize=14)
plt.title('Number of Lines', fontsize=18)
plt.legend([])
plt.show()

In [ ]:
# stacked histogram showing percentage answered

length_vs_response = pd.crosstab([df_lines_of_code], # categories to cross tabulate
                      df.Duration[ df.Code.notnull() ].isnull()) # how to group
length_vs_response.plot(kind='barh', title="Questions Answered by Code Length", stacked=True)
plt.legend(["Answered", "Unanswered"])
plt.show()

In [ ]:
# https://stackoverflow.com/questions/43606339/generate-word-cloud-from-single-column-pandas-dataframe
from wordcloud import WordCloud

wordcloud = WordCloud(
    background_color='white',
    width=300,
    height=200,
    max_font_size=40).generate(' '.join(df_tags.Tag))
plt.figure(figsize=(8,10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()